In [ ]:
import os
import numpy as np
import time
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from Common import NeuralNet, fit, FNO2d

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
##### GLOBAL PARAMETERS #####

d = 3

x1_min, x1_max = -1.0, 1.0
x2_min, x2_max = -1.0, 1.0

mu_min, mu_max = -1.0, 1.0

final_time = 0.1

# Define the initial condition function
def u0(input_vector):
    u = 0
    for m in range(1, d+1):
        u -= input_vector[2+m-1] * np.sin(np.pi * m * input_vector[0]) * np.sin(np.pi * m * input_vector[1]) / np.sqrt(m)
    return u / d

# Define the solution function
def u(t, input_vector):
    u = 0
    for m in range(1, d+1):
        u -= np.exp(-(np.pi * m) ** 2 * t) * input_vector[2+m-1] * np.sin(np.pi * m * input_vector[0]) * np.sin(np.pi * m * input_vector[1]) / np.sqrt(m)
    return u / d

# Parametric approach

### Data

In [ ]:
##### GENERATE TRAINING INPUT #####

num_positions = 32**2
num_params = 32**d

# positions

soboleng_positions = torch.quasirandom.SobolEngine(dimension=2, scramble=True)
sobol_positions = soboleng_positions.draw(num_positions).numpy()

sobol_positions[:, 0] *= (x1_max-x1_min)
sobol_positions[:, 0] += x1_min 
sobol_positions[:, 1] *= (x2_max-x2_min) 
sobol_positions[:, 1] += x2_min

# parameters

soboleng_params = torch.quasirandom.SobolEngine(dimension=d, scramble=True)
sobol_params = soboleng_params.draw(num_params).numpy()
# sobol_params = torch.rand((num_params, d))

for m in range(d):
    sobol_params[:, m] = sobol_params[:, m] * (mu_max-mu_min) + mu_min 

training_inputs = np.zeros((num_positions*num_params, 2+d))
training_inputs[:, 0] = np.tile(sobol_positions[:, 0], num_params)
training_inputs[:, 1] = np.tile(sobol_positions[:, 1], num_params)
for m in range(d):
    training_inputs[:, 2+m] = np.tile(sobol_params[:, m], num_positions).reshape(num_positions, num_params).transpose().flatten()

print(training_inputs.shape)
np.save("data/training_inputs.npy", training_inputs)


##### GENERATE TRAINING OUTPUT #####

training_outputs = u(final_time, training_inputs.transpose())
print(training_outputs.shape)
np.save("data/training_outputs.npy", training_outputs)

In [ ]:
input_train = np.load("data/training_inputs.npy")
output_train = np.load("data/training_outputs.npy")

print(input_train.shape, output_train.shape)

in_test = input_train[:num_positions, :]
X_test, Y_test = in_test[:, 0], in_test[:, 1]
out_test = output_train[:num_positions]
_min, _max = np.min(out_test), np.max(out_test)

fig = plt.figure(figsize=(6, 5))

ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_trisurf(X_test, Y_test, out_test, cmap=cm.jet, linewidth=0.1)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_title('Solution at final time')

### Model

In [ ]:
##### FINAL TRAINING DATA RETYPING #####

input_train_torch = torch.tensor(input_train, dtype=torch.float32)
output_train_torch = torch.tensor(output_train, dtype=torch.float32)
input_train_torch = input_train_torch.to(device)
output_train_torch = output_train_torch.to(device)
training_dataset = TensorDataset(torch.tensor(input_train_torch), torch.tensor(output_train_torch))
training_dataset_dl = DataLoader(training_dataset, batch_size=2048, shuffle=True)

##### MODEL DEFINITION #####

neurons_per_layer = 26
num_hidden_layers = 3

model = NeuralNet(
    input_dimension=2+d,
    output_dimension=1,
    n_hidden_layers=num_hidden_layers,
    neurons=neurons_per_layer,
    regularization_exp=0,
    regularization_param=0,
    retrain_seed=1
    )
model = model.to(device)
pytorch_total_params_parametric = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("number of trainable parameters:", pytorch_total_params_parametric)
##### TRAINING #####

num_weights = (num_hidden_layers-1)*(neurons_per_layer+1)*neurons_per_layer + (2+d+1)*neurons_per_layer + (neurons_per_layer+1)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("number of degrees of freedom:", num_weights, pytorch_total_params)


learning_rate = 2e-3
num_epochs = 2

optimizer_ADAM = optim.Adam(
    model.parameters(),
    lr=float(learning_rate)
    )

history, mse, epoch_time = fit(
    model=model,
    training_set=training_dataset_dl,
    num_epochs=num_epochs,
    optimizer=optimizer_ADAM,
    p=2,
    verbose=True
)

In [ ]:
# Plot history vs epoch and history vs epoch * epoch_time using subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot history vs epoch
axes[0].plot(np.arange(len(mse)) + 1, mse)
axes[0].set_yscale('log')
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("MSE Loss vs Epoch")

# Plot history vs epoch * epoch_time

axes[1].plot((np.arange(len(mse)) + 1) * epoch_time, mse)
axes[1].set_xlabel("Time (s)")
axes[1].set_yscale('log')
axes[1].set_ylabel("Loss")
axes[1].set_title("MSE Loss vs Time")
# plt.ylim([5e-4, 2e-1 ])
plt.tight_layout()
plt.show()
print("Average epoch time =  " + str(epoch_time))

### Visualize

In [ ]:
##### GENERATE TESTING DATA #####

num_positions_test = 10000
num_params_test = 1
model = model.cpu()
# positions

positions_test = np.random.random((num_positions_test,2))

positions_test[:, 0] *= (x1_max-x1_min)
positions_test[:, 0] += x1_min 
positions_test[:, 1] *= (x2_max-x2_min) 
positions_test[:, 1] += x2_min

# parameters

mu_test = torch.rand((1,d))
for m in range(d):
    mu_test[:, m] *= (mu_max-mu_min) 
    mu_test[:, m] += mu_min 
print("test parameters:", mu_test)

test_inputs = np.zeros((num_positions_test*num_params_test, 2+d))
test_inputs[:, 0] = np.tile(positions_test[:, 0], num_params_test)
test_inputs[:, 1] = np.tile(positions_test[:, 1], num_params_test)
for m in range(d):
    test_inputs[:, 2+m] = np.tile(mu_test[:, m], num_positions_test).reshape(num_positions_test, num_params_test).transpose().flatten()

prediction = np.array([model.forward(torch.tensor(test_inputs[i, :], dtype=torch.float32)).detach().item() for i in range(num_positions_test)])
ground_truth = u(final_time, test_inputs.transpose())

fig = plt.figure(figsize=(18, 5))

ax1 = fig.add_subplot(131)
# surf = ax.plot_trisurf(test_inputs[:, 1], test_inputs[:, 1], prediction, cmap=cm.jet, linewidth=0.1)
# fig.colorbar(surf, shrink=0.5, aspect=5)
im1 = ax1.scatter(test_inputs[:, 0], test_inputs[:, 1], c=prediction, cmap='viridis')
plt.colorbar(im1)
ax1.set_xlabel('X1')
ax1.set_ylabel('X2')
ax1.set_title('NN Prediction')

ax2 = fig.add_subplot(132)
# surf = ax.plot_trisurf(test_inputs[:, 1], test_inputs[:, 1], ground_truth, cmap=cm.jet, linewidth=0.1)
# fig.colorbar(surf, shrink=0.5, aspect=5)
im2 = ax2.scatter(test_inputs[:, 0], test_inputs[:, 1], c=ground_truth, cmap='viridis')
plt.colorbar(im2)
ax2.set_xlabel('X1')
ax2.set_ylabel('X2')
ax2.set_title('Ground truth')

ax3 = fig.add_subplot(133)
# surf = ax.plot_trisurf(test_inputs[:, 1], test_inputs[:, 1], ground_truth, cmap=cm.jet, linewidth=0.1)
# fig.colorbar(surf, shrink=0.5, aspect=5)
im3 = ax3.scatter(test_inputs[:, 0], test_inputs[:, 1], c=np.abs(ground_truth-prediction), cmap='viridis')
plt.colorbar(im3)
ax3.set_xlabel('X1')
ax3.set_ylabel('X2')
ax3.set_title('Error')

# Operator approach

### Model


In [ ]:
discretization = 32

x1_points = torch.linspace(x1_min, x1_max, discretization)
x2_points = torch.linspace(x2_min, x2_max, discretization)

position_grid = torch.stack(torch.meshgrid(x1_points, x2_points),axis=2).reshape((discretization, discretization, 2)).transpose(0, 1)

d = 2
n_samples = 32**d

param_samples = torch.rand((n_samples, d))*(mu_max - mu_min) + mu_min

fno_input_positions = torch.tile(position_grid.reshape((1, discretization**2, 2)), (n_samples, 1, 1))
# print(fno_input_positions.shape)
fno_input_params = torch.tile(param_samples.reshape(n_samples, 1, d), (1, discretization**2, 1))
# print(fno_input_params.shape)
fno_input_vectors = torch.concatenate((fno_input_positions, fno_input_params), axis=-1).reshape(n_samples*discretization*discretization, 2+d)
# print(fno_input_vectors.shape)

fno_input_train = torch.concatenate(
    (
        u0(np.transpose(fno_input_vectors)).reshape((n_samples, discretization, discretization, 1)), 
        fno_input_vectors[:, 0].reshape((n_samples, discretization, discretization, 1)),
        fno_input_vectors[:, 1].reshape((n_samples, discretization, discretization, 1))
    ), 
    axis=-1)
fno_output_train = u(final_time, np.transpose(fno_input_vectors)).reshape((n_samples, discretization, discretization, 1))

print("input:", fno_input_train.shape)
print("output:", fno_output_train.shape)

# np.savetxt("fno_input_vectors.csv", np.array(fno_input_vectors.detach()), delimiter=',')

n_train = int(0.8*n_samples)
print("n_train = ", n_train)

input_function_train = fno_input_train[:n_train, :, :, :]
output_function_train = fno_output_train[:n_train, :, :, :]
input_function_test = fno_input_train[n_train:, :, :, :]
output_function_test = fno_output_train[n_train:, :, :, :]

batch_size = 64

training_set = DataLoader(TensorDataset(input_function_train, output_function_train), batch_size=batch_size, shuffle=False)
testing_set = DataLoader(TensorDataset(input_function_test, output_function_test), batch_size=batch_size, shuffle=False)

print(input_function_train.shape)
print(output_function_train.shape)
print(input_function_test.shape)
print(output_function_test.shape)

In [ ]:
learning_rate = 0.0002

epochs = 10
step_size = 20
gamma = 0.5

modes = 2
width = 8
n_layers = 2

fno_architecture = {
    "modes": modes,
    "width": width,
    "n_layers": n_layers,
    "retrain_fno": 1
}


fno = FNO2d(fno_architecture)

pytorch_total_params_fno = sum(p.numel() for p in fno.parameters() if p.requires_grad)
print("number of trainable parameters:", pytorch_total_params_fno)

num_weights = (3+1)*width + n_layers*((width+1)*width + 2*width*width*modes*modes) + (width+1)*128 + (128+1)
pytorch_total_params = sum(p.numel() for p in fno.parameters() if p.requires_grad)
print("number of degrees of freedom:", num_weights, pytorch_total_params)

optimizer = optim.Adam(fno.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

train_loss_mse = []
test_loss_l2 = []
epoch_time_list = []


l = torch.nn.MSELoss()
freq_print = 1
for epoch in range(epochs):
    t0 = time.time()
    train_mse = 0.0
    for step, (input_batch, output_batch) in enumerate(training_set):
        optimizer.zero_grad()
        output_pred_batch = fno(input_batch).squeeze(2)
        loss_f = l(output_pred_batch, output_batch)
        loss_f.backward()
        optimizer.step()
        train_mse += loss_f.item()
    train_mse /= len(training_set)
    train_loss_mse.append(train_mse)

    scheduler.step()

    with torch.no_grad():
        fno.eval()
        test_relative_l2 = 0.0
        for step, (input_batch, output_batch) in enumerate(testing_set):
            output_pred_batch = fno(input_batch).squeeze(2)
            loss_f = (torch.mean((output_pred_batch - output_batch) ** 2) / torch.mean(output_batch ** 2)) ** 0.5 * 100
            test_relative_l2 += loss_f.item()
        test_relative_l2 /= len(testing_set)
        test_loss_l2.append(test_relative_l2)
    ep_time = time.time() - t0
    epoch_time_list.append(ep_time)
    if epoch % freq_print == 0: print("######### Epoch:", epoch, " ######### Train Loss:", train_mse, " ######### Relative L2 Test Norm:", test_relative_l2)
time_x = []
for i in range(len(epoch_time_list)):
    time_x.append(sum(epoch_time_list[:i]))


In [ ]:
fig, (ax1,ax2, ax3) = plt.subplots(1,3,figsize=(18,6))
plt.ylim([5e-4, 2e-1 ])
ax1.plot(train_loss_mse)
ax1.set_title("Training loss (MSE) vs epoch")
ax1.set_yscale('log')
ax2.plot(test_loss_l2)
ax2.set_title("Test loss (L2)")
ax2.set_yscale('log')
ax3.plot(time_x, train_loss_mse)
ax3.set_title("Training loss (MSE) vs Time")
ax3.set_yscale('log')
print("Average epoch time =  " + str(np.average(np.array(epoch_time_list))))

### Visualize

In [ ]:
# ON A TRAINING SAMPLE

k = 3
print(param_samples[k, :])

test_input = fno_input_train[k:k+1, :, :, :]

prediction = fno.forward(test_input).reshape((discretization, discretization))
ground_truth = fno_output_train[k, :, :, :].reshape((discretization, discretization))

print(prediction.shape)
print(ground_truth.shape)

xx, yy = np.meshgrid(x1_points, x2_points)

fig = plt.figure(figsize=(18, 5))

ax1 = fig.add_subplot(131)

im1 = ax1.scatter(xx, yy, c=prediction.detach().numpy(), cmap='viridis')
ax1.set_title("FNO prediction")
plt.colorbar(im1)
ax2 = fig.add_subplot(132)
im2 = ax2.scatter(xx, yy, c=ground_truth.detach().numpy(), cmap='viridis')
ax2.set_title("Ground truth")
plt.colorbar(im2)
ax3 = fig.add_subplot(133)
im3 = ax3.scatter(xx, yy, c=np.abs(ground_truth.detach().numpy()-prediction.detach().numpy()), cmap='viridis')
ax3.set_title("Error")
plt.colorbar(im3)

In [ ]:
# ON A NEW SAMPLE

n_samples = 1

param_samples = torch.rand((n_samples, d))*(mu_max - mu_min) + mu_min
print(param_samples)

fno_input_positions = torch.tile(position_grid.reshape((1, discretization**2, 2)), (n_samples, 1, 1))
# print(fno_input_positions.shape)
fno_input_params = torch.tile(param_samples.reshape(n_samples, 1, d), (1, discretization**2, 1))
# print(fno_input_params.shape)
fno_input_vectors = torch.concatenate((fno_input_positions, fno_input_params), axis=-1).reshape(n_samples*discretization*discretization, 2+d)
# print(fno_input_vectors.shape)

fno_input_train = torch.concatenate(
    (
        u0(np.transpose(fno_input_vectors)).reshape((n_samples, discretization, discretization, 1)), 
        fno_input_vectors[:, 0].reshape((n_samples, discretization, discretization, 1)),
        fno_input_vectors[:, 1].reshape((n_samples, discretization, discretization, 1))
    ), 
    axis=-1)
fno_output_train = u(final_time, np.transpose(fno_input_vectors)).reshape((n_samples, discretization, discretization, 1))


prediction = fno.forward(fno_input_train).reshape((discretization, discretization))
ground_truth = fno_output_train[0, :, :, :].reshape((discretization, discretization))

print(prediction.shape)
print(ground_truth.shape)

xx, yy = np.meshgrid(x1_points, x2_points)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,5))

im1 = ax1.scatter(xx, yy, c=prediction.detach().numpy(), cmap='viridis')
ax1.set_title("FNO prediction")
plt.colorbar(im1)
im2 = ax2.scatter(xx, yy, c=ground_truth.detach().numpy(), cmap='viridis')
ax2.set_title("Ground truth")
plt.colorbar(im2)
im3 = ax3.scatter(xx, yy, c=np.abs(ground_truth.detach().numpy()-prediction.detach().numpy()), cmap='viridis')
ax3.set_title("Error")
plt.colorbar(im3)